In [ ]:
# Dom's Helper Code was of great help through this homework
import os
# Find the latest version of spark 3.2.x  from http://www.apache.org/dist/spark/ and enter as the spark version
spark_version = 'spark-3.2.3'

os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Hit:3 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Get:5 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease [1,581 B]
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [83.3 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:13 http://security.ubuntu.c

In [ ]:
# Install the PostgreSQL driver in our Colab environment
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2022-11-29 01:54:30--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar’

postgresql-42.2.9.j 100%[===================>] 892.61K  5.73MB/s    in 0.2s    

2022-11-29 01:54:30 (5.73 MB/s) - ‘postgresql-42.2.9.jar’ saved [914037/914037]



In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Digital_Music").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [ ]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Digital_Music_Purchase_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
music_df = spark.read.csv(SparkFiles.get("amazon_reviews_us_Digital_Music_Purchase_v1_00.tsv.gz"), sep="\t", header=True, inferSchema=True)

# Show DataFrame
music_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+--------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|    product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+--------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   10293599|R14LVB34Z2Z53I|B000YMOQZY|     262935067|Knockin' On Heave...|Digital_Music_Pur...|          5|            0|          0|   N|                Y|           favorites|Great  rendition....| 2015-08-31|
|         US|    6087195|R2M0R9CWPC82LT|B00ISAEC80|     452318038|            Flawless|Digital_Music_Pur

In [ ]:
rows = music_df.count()
print(f"DataFrame Rows count : {rows}")

DataFrame Rows count : 1688884


In [ ]:
usable_df = music_df.select('customer_id','review_id','product_id','product_parent','product_title','star_rating','helpful_votes','total_votes','vine','review_date')
usable_df.show()

+-----------+--------------+----------+--------------+--------------------+-----------+-------------+-----------+----+-----------+
|customer_id|     review_id|product_id|product_parent|       product_title|star_rating|helpful_votes|total_votes|vine|review_date|
+-----------+--------------+----------+--------------+--------------------+-----------+-------------+-----------+----+-----------+
|   10293599|R14LVB34Z2Z53I|B000YMOQZY|     262935067|Knockin' On Heave...|          5|            0|          0|   N| 2015-08-31|
|    6087195|R2M0R9CWPC82LT|B00ISAEC80|     452318038|            Flawless|          5|            0|          0|   N| 2015-08-31|
|   33717063| RQNQI2R3BM52A|B00VBB6AVU|     675969129|    Scandal of Grace|          4|            0|          0|   N| 2015-08-31|
|   14948020| RTUIX1135HUAB|B008A5POJO|     282925893|  I Surrender (Live)|          5|            0|          0|   N| 2015-08-31|
|   21694522| RE3QSAT8TL010|B014CW2RN4|     109077793|We Are Young (Jer...|        

In [ ]:
# Huge Shout out to Brandon Groenewold for this line of code
clean_customer_df = usable_df.groupby("customer_id").agg({"customer_id":"count"})
clean_customer_df.show()

+-----------+------------------+
|customer_id|count(customer_id)|
+-----------+------------------+
|   39290568|                 1|
|   23601866|                 1|
|   19004376|                 1|
|   16320823|                 1|
|   13973119|                 1|
|    1155154|                 1|
|   24317024|                 2|
|   15125275|                 1|
|   35781732|                 2|
|   40985731|                12|
|   19522542|                 1|
|   50243180|                 1|
|    2702985|                 1|
|   41857916|                 1|
|    1943933|                 2|
|   42050930|                 5|
|    4867895|                 1|
|   46317087|                 3|
|   52021773|                 2|
|   15504622|                 9|
+-----------+------------------+
only showing top 20 rows



In [ ]:
clean_customer_df = clean_customer_df.withColumnRenamed('count(customer_id)', 'customer_count')
clean_customer_df.show()

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|   39290568|             1|
|   23601866|             1|
|   19004376|             1|
|   16320823|             1|
|   13973119|             1|
|    1155154|             1|
|   24317024|             2|
|   15125275|             1|
|   35781732|             2|
|   40985731|            12|
|   19522542|             1|
|   50243180|             1|
|    2702985|             1|
|   41857916|             1|
|    1943933|             2|
|   42050930|             5|
|    4867895|             1|
|   46317087|             3|
|   52021773|             2|
|   15504622|             9|
+-----------+--------------+
only showing top 20 rows



In [ ]:
clean_products_df = usable_df.select('product_id','product_title')
clean_products_df = clean_products_df.dropDuplicates()
clean_products_df.show()

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|B014PO1708|       See You Again|
|B001APIS9Y|What Happened to ...|
|B005XT4CAW|I Can Dream About...|
|B006WH9FPQ|  If You Want To Fly|
|B0026G2MG6|Ghetto Fabolous [...|
|B0096RUQZW|       Call Me Maybe|
|B003YZGVFQ|Maxwell's silver ...|
|B00138B7N2|He Touched Me (Al...|
|B00OLF0U3M|Beyond The Lights...|
|B007D89P76|A Hampshire Suite...|
|B0011U5E1S|             Nirvana|
|B00JWFMVNS|Sousa (Red Classics)|
|B00120EDBY|Talkin' To You Ey...|
|B00IWI3HNO|          Sweet Life|
|B00YIB4KJY|                Fall|
|B0011ZVFIO|Carmel City (LP V...|
|B00C32RB4Q|         Lola Montez|
|B001NTZ4NA|Thug Lovin' (Albu...|
|B001BHJZN4|      Eye In The Sky|
|B005SR8QO2|      My Kinda Party|
+----------+--------------------+
only showing top 20 rows



In [ ]:
# Huge thanks to Arnold Schultz for the to_date
from pyspark.sql.functions import to_date
clean_review_df = usable_df.select('review_id','customer_id','product_id','product_parent', to_date('review_date','yyyy-mm-dd').alias('review_date'))
clean_review_df.show()

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
|R14LVB34Z2Z53I|   10293599|B000YMOQZY|     262935067| 2015-01-31|
|R2M0R9CWPC82LT|    6087195|B00ISAEC80|     452318038| 2015-01-31|
| RQNQI2R3BM52A|   33717063|B00VBB6AVU|     675969129| 2015-01-31|
| RTUIX1135HUAB|   14948020|B008A5POJO|     282925893| 2015-01-31|
| RE3QSAT8TL010|   21694522|B014CW2RN4|     109077793| 2015-01-31|
|R14TN65IY0NUOD|   14948020|B00DRA3EOY|     959245080| 2015-01-31|
|R3LIF8MCNH1ODF|   20031966|B009G3U0PW|     762436916| 2015-01-31|
|R33X0DGA4PPQ3L|   12223745|B013XBYQOS|     137980344| 2015-01-31|
|R2DJJP7DQRT1ZW|   30231876|B003L5H25W|     165995375| 2015-01-31|
|R2A8XDXW5XOT4H|   51714365|B00136NVH4|     227508666| 2015-01-31|
|R3MJK3L7ER61TT|   37216001|B0030AKPQ6|     164096943| 2015-01-31|
|R38Z7XMCVDR0R5|   49207156|B014EQE0V0|     714368268| 2015-01

In [ ]:
clean_vine_df = usable_df.select('review_id','star_rating','helpful_votes','total_votes','vine')
clean_vine_df.show()

+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
|R14LVB34Z2Z53I|          5|            0|          0|   N|
|R2M0R9CWPC82LT|          5|            0|          0|   N|
| RQNQI2R3BM52A|          4|            0|          0|   N|
| RTUIX1135HUAB|          5|            0|          0|   N|
| RE3QSAT8TL010|          4|            0|          0|   N|
|R14TN65IY0NUOD|          5|            0|          0|   N|
|R3LIF8MCNH1ODF|          5|            0|          0|   N|
|R33X0DGA4PPQ3L|          5|            3|          3|   N|
|R2DJJP7DQRT1ZW|          5|            0|          1|   N|
|R2A8XDXW5XOT4H|          5|            0|          0|   N|
|R3MJK3L7ER61TT|          5|            0|          0|   N|
|R38Z7XMCVDR0R5|          5|            1|          1|   N|
|R3FFESBQXPR7S6|          4|            0|          0|   N|
|R3QO6Z942CKH34|          5|            

In [ ]:
# IMPORTANT: Replace each of these parameters with your own values for your AWS RDS instance
my_aws_endpoint = 'grimsbear-aws-database.ci78mzho4eyo.us-east-1.rds.amazonaws.com' # This is my value; please replace with your own
my_aws_port_number = '5432' # Your value is likely the same, but please double check
my_aws_database_name = 'my_data_class_db' # This is my value; please replace with your own
my_aws_username = 'postgres' # Your value is likely the same, but please double check
my_aws_password = '' # This is my value; please replace with your own

In [ ]:
# Define the connection string
jdbc_url=f'jdbc:postgresql://{my_aws_endpoint}:{my_aws_port_number}/{my_aws_database_name}'

# Set up the configuration parameters
config = {"user": f'{my_aws_username}', 
          "password": f'{my_aws_password}', 
          "driver":"org.postgresql.Driver"}

# Choose to overwrite the existing data. Note that 'append' is probably a smarter choice
# in those situations where the PostgreSQL schema automatically generates the primary key. 
#  But 'overwrite' works best for this little demo. 
mode = 'append' 

In [ ]:
# Write the dataframe to the appropriate table in your PostgreSQL RDS

clean_customer_df.write.jdbc(url=jdbc_url, table='customers', mode=mode, properties=config)

Py4JJavaError: ignored

In [ ]:
# Write the dataframe to the appropriate table in your PostgreSQL RDS

clean_products_df.write.jdbc(url=jdbc_url, table='products', mode=mode, properties=config)

In [ ]:
# Write the dataframe to the appropriate table in your PostgreSQL RDS

clean_review_df.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)

In [ ]:
# Write the dataframe to the appropriate table in your PostgreSQL RDS

clean_vine_df.write.jdbc(url=jdbc_url, table='vine_table', mode=mode, properties=config)